Neste Notebook o dataset dos vistos disponível na internet, pelo link abaixo, e que contem mais de 3 milhões de linhas é modificado para um novo dataset. 

https://www.kaggle.com/nsharan/h-1b-visa

In [3]:
import pandas as pd
import numpy as np
import random
from pandas import ExcelWriter

Na célula abaixo, o dataset é importado e após isso a ordem de todas suas linhas são embaralhadas. 

In [4]:
dataset = pd.read_csv("h1b.csv")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2696365,CERTIFIED,VALLOUREC INDUSTRIES INC.,Management Analysts,LEAD SAP ANALYST,Y,71822.0,2011.0,"YOUNGSTOWN, OHIO",-80.649519,41.099780
1,1332301,CERTIFIED,WICHITA STATE UNIVERSITY,"Financial Specialists, All Other",TRAVEL SERVICES AUDITOR & COMPLIANCE ANALYST,Y,28912.0,2014.0,"WICHITA, KANSAS",-97.330053,37.687176
2,1801297,CERTIFIED,"TWIN PEAKS, INC",Civil Engineers,CIVIL ENGINEER,Y,61297.6,2013.0,"LONG ISLAND CITY, NEW YORK",-73.948542,40.744679
3,760656,CERTIFIED,NORTHBRIDGE LLC,SOCIAL AND COMMUNITY SERVICE MANAGERS,PROGRAM DIRECTOR,Y,54558.0,2015.0,"ANCHORAGE, ALASKA",-149.900278,61.218056
4,2338494,CERTIFIED,GREY GLOBAL GROUP INC.,Advertising and Promotions Managers,PLANNING DIRECTOR,Y,181688.0,2012.0,"NEW YORK, NEW YORK",-74.005941,40.712784


Em primeiro lugar o objetivo do estudo, CASE_STATUS. As opções possívies de resposta são: CERTIFIED, DENIED, WITHDRAWN, CERTIFIED-WITHDRAWN, INVALIDATED, PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED. 
CERTIFIED quando a pessoa é aprovada, DENIED é quando o visto é negado. Já WITHDRAWN corresponde ao caso quando a pessoa que pediu o visto interrompe o processo antes de ele ser terminado, nesse caso, não é possível determinar se a pessoa foi aprovada ou não, então este caso é desconsiderado para nosso estudo. CERTIFIED-WITHDRAWN, muito parecido com o WITHDRAWN é quando a pessoa interrompe o processo antes dele acabar porém a pessoa já foi aprovada, ou seja a pessoa teria obtido o visto porém resolveu não seguir com o processo. Para o nosso estudo não importa se a pessoa acabou o processo ou não, apenas se ela teria sido aprovada ou não, então resolvemos juntar CERTIFIED-WITHDRAWN com CERTIFIED, os casos de aprovação. Por último, INVALIDATED e PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED, são casos em que a pessoa não preencheu a ficha de pedido de visto correteamente ou foi descualificada por algum outro motivo. Para estes casos, novamente não é possível determinar se a pessoa foi aprovada ou não, por isso também acabaram desconsiderados. 
Essas processo todo de descartar e juntar categorias, descrito acima é feito na célula abaixo. Nas primeiras três primeiras linhas de código as categorias desnecessárias são removidas do dataset. Na linha de código seguinte as variavés são transformadas em 0 ou 1, para quando aprovado ou reprovado (CERTIFIED OU DENIED). Por último todas linhas do dataset que não haviam uma das colunas completas, são removidas.

In [5]:
dataset = dataset[dataset.CASE_STATUS != "WITHDRAWN"]
dataset = dataset[dataset.CASE_STATUS != "PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED"]
dataset = dataset[dataset.CASE_STATUS != "INVALIDATED"]

dataset["CASE_STATUS"] = dataset["CASE_STATUS"].apply(lambda x: 1 if (x=="CERTIFIED-WITHDRAWN" or x=="CERTIFIED") else 0)

dataset = dataset.dropna()
dataset.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2696365,1,VALLOUREC INDUSTRIES INC.,Management Analysts,LEAD SAP ANALYST,Y,71822.0,2011.0,"YOUNGSTOWN, OHIO",-80.649519,41.099780
1,1332301,1,WICHITA STATE UNIVERSITY,"Financial Specialists, All Other",TRAVEL SERVICES AUDITOR & COMPLIANCE ANALYST,Y,28912.0,2014.0,"WICHITA, KANSAS",-97.330053,37.687176
2,1801297,1,"TWIN PEAKS, INC",Civil Engineers,CIVIL ENGINEER,Y,61297.6,2013.0,"LONG ISLAND CITY, NEW YORK",-73.948542,40.744679
3,760656,1,NORTHBRIDGE LLC,SOCIAL AND COMMUNITY SERVICE MANAGERS,PROGRAM DIRECTOR,Y,54558.0,2015.0,"ANCHORAGE, ALASKA",-149.900278,61.218056
4,2338494,1,GREY GLOBAL GROUP INC.,Advertising and Promotions Managers,PLANNING DIRECTOR,Y,181688.0,2012.0,"NEW YORK, NEW YORK",-74.005941,40.712784


Em seguida é necessário, para treinar nosso modelo, separar uma certa quantia de casos que foram aprovados e caso que foram reprovados, no nosso caso escolhemos 50 000 de cada. 
Para isso o dataset e duplicado em dois novos dataset, um contém apenas aprovações e o outro reprovações (as duas primeiras linhas de código). Em seguida esse dois datasets são recortados para o valor escolhido, nas três linhas de código seguinte.
Após o corte, os dois datasets são juntados em apenas um, e embaralhado para que não estejam todos os aprovados juntos e nem todos reprovados, duas últimas linhas de código. E por fim, na última célula o dataset é exportado para um arquivo ".csv" .

In [6]:
na = dataset[dataset["CASE_STATUS"] == 0]
ap = dataset[dataset["CASE_STATUS"] == 1]

size=50000
na_cut = na[:size]
ap_cut = ap[:size]

df = pd.concat([ap_cut, na_cut])
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,2297770,0,"FLOWCON AMERICAS, LLC","Managers, All Other",SUPPLY CHAIN MANAGER,Y,63128.0,2012.0,"CHARLOTTE, NORTH CAROLINA",-80.843127,35.227087
1,2447581,1,"FMC TECHNOLOGIES, INC.",Mechanical Engineers,SYSTEMS ENGINEER,Y,92120.0,2012.0,"HOUSTON, TEXAS",-95.369803,29.760427
2,1668413,1,EVANTAGE SOLUTIONS INC.,"Software Developers, Applications",SENIOR JAVA DEVELOPER,Y,87422.0,2014.0,"WASHINGTON, DISTRICT OF COLUMBIA",-77.036871,38.907192
3,2595250,1,"BIRMINGHAM BUILDERS, INC.","Property, Real Estate, and Community Associati...",PROPERTY MANAGER,Y,26499.0,2012.0,"SAN FRANCISCO, CALIFORNIA",-122.419415,37.774929
4,1794371,1,SAPIENT CORPORATION,Management Analysts,"DIRECTOR, PROGRAM MANAGEMENT",Y,97552.0,2013.0,"MT LAUREL, NEW JERSEY",-74.890999,39.934002


In [7]:
df.to_csv("Dataframe.csv")